In [ ]:
import geopandas as gpd
from ipyfilechooser import FileChooser
import numpy as np
from pathlib import Path
import rioxarray as rxr
import rioxarray.merge
from tqdm.auto import tqdm
from typing import List

import s3fs
from shapely.geometry import mapping

### Select the chip shape file

In [ ]:
print("Select the chip shape file generated in Notebook 0")
fc = FileChooser(Path.cwd())
display(fc)

### Load the chip geometry from the chip shapefile

In [ ]:
shape_path = Path(fc.selected)
shape_path

In [ ]:
geometry_gdf = gpd.read_file(shape_path)
geometry = geometry_gdf['geometry'][0]
geometry

### Collect the chip directory and ID

In [ ]:
chip_dir = shape_path.parent
chip_id = chip_dir.name.split('_')[-1]
chip_dir

### Load the S1 class zones into a dataframe

In [ ]:
df_zones = gpd.read_file('s2_class_zones.geojson')
df_zones.head()

### Create a dataframe of tiles intersecting the chip

In [ ]:
intersections = df_zones.geometry.intersects(geometry)
s1_zones = df_zones[intersections].reset_index(drop=True)
s1_zones

### Download tiles and clip them to chip geometry

In [ ]:
ds_url = "https://lulctimeseries.blob.core.windows.net/lulctimeseriespublic/lc2021/"
ds_date_range = "20210101-20220101"

urls = [f"{ds_url}{zone}_{ds_date_range}.tif" for zone in s1_zones.zone]
urls

In [ ]:
to_merge = list()
for url in urls:
    print(url)
    tile = rxr.open_rasterio(url, masked=True).squeeze()
    tile = tile.rio.clip(geometry_gdf.geometry.apply(mapping), 'EPSG:4326')
    to_merge.append(tile)
to_merge

### Merge clipped tiles and output to GeoTiff

In [ ]:
merged = rxr.merge.merge_arrays(to_merge)
merged.rio.to_raster(chip_dir/f"S1_LULC_{chip_id}.tif")